# 주요 질병 사망자 및 보건 환경 비교 분석
## - 6 - 종합 및 결론
(Comparative Analysis of Major Disease Deaths and Health Environment)
## 1. 소개
국가별 의료 관련 데이터들을 이용해, 국가별, 연간 질병 사망자 수와 의료 인프라 간의 관계를 분석해보고 시각화를 해본 프로젝트

## 2. 목표
- 각 데이터의 EDA와 시각화
- 비슷한 분류의 데이터를 하나로 합치기
- 의료 인프라와 질병 사망자 수 관계 찾기 및 분석

## 3. 데이터 셋
- [KOSIS 10만명당 주요사망원인별 사망률](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&conn_path=I2)
- [KOSIS 의료종사자수](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC01&conn_path=I2)
- [KOSIS 보건관련 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC09&conn_path=I2)
- [KOSIS 보건서비스 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC14&conn_path=I2)
- [KOSIS 의약품 판매](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC25&conn_path=I2)
- [KOSIS 의약품 소비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC24&conn_path=I2)
- [KOSIS 공공사회복지 지출](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAD21&conn_path=I2)
- [geojson 데이터](https://github.com/dr5hn/countries-states-cities-database/blob/master/countries.json)

## 4. 구현

#### 라이브러리

In [1]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import cufflinks as cf
import chart_studio
import folium
from folium.plugins import MarkerCluster
import json

#### 한글 폰트 설정

In [2]:
from IPython.display import set_matplotlib_formats

def get_font_family():
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

get_font_family()

plt.style.use("ggplot")

font_family = get_font_family()
plt.rc("font", family=font_family)
plt.rc("axes", unicode_minus=False)

set_matplotlib_formats("retina")

C:\Users\spec3\AppData\Local\Temp\ipykernel_37124\2328754805.py:32: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [3]:
# 모든 데이터 공통 국가
country_intersection = ['룩셈부르크',  '네덜란드',  '영국',  '이탈리아',  '캐나다',  '오스트레일리아',  '한국',  '일본',  '스페인',  '헝가리',  '독일',  '에스토니아',  '그리스',  '슬로바키아',  '핀란드',  '벨기에',  '체코',  '슬로베니아',  '프랑스',  '스웨덴',  '노르웨이',  '뉴질랜드',  '라트비아',  '덴마크',  '오스트리아',  '포르투갈',  '아일랜드',  '아이슬란드']

#### Data Load
각 파트별로 EDA 및 시각화를 진행한 후, 최종적으로 전처리된 파일은 6개입니다.  
**종합 및 결론**에서 사용하는 데이터 파일은 다음과 같이 총 8개 입니다.
- [KOSIS 10만명당 주요사망원인별 사망률](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_death_rate`
- [KOSIS 의료종사자수](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_Nmw`
- [KOSIS 보건관련 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_service`
- [KOSIS 보건서비스 지출비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_service_common`
- [KOSIS 의약품 판매](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_medicion`
- [KOSIS 의약품 소비](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_medicion`
- [KOSIS 공공사회복지 지출](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAC13&vw_cd=MT_RTITLE&list_id=101_003_E&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_RTITLE) → `df_welfare`
- [geojson 데이터](https://github.com/dr5hn/countries-states-cities-database/blob/master/countries.json) → `countries.geo.edited.json`, `countries.json`

In [4]:
df_death_rate = pd.read_csv("data/pre_df/df_death_rate.csv")
df_Nmw = pd.read_csv("data/pre_df/df_Nmw.csv")
df_service = pd.read_csv("data/pre_df/df_service.csv")
df_service_common = pd.read_csv("data/pre_df/df_service_common.csv")
df_medicion = pd.read_csv("data/pre_df/df_medicion.csv")
df_welfare = pd.read_csv("data/pre_df/df_welfare.csv")

In [5]:
geo_poly = open("data/countries.geo.edited.json", encoding="utf-8")
gp = open("data/countries.json", encoding="utf-8")
geo_point = pd.json_normalize(json.load(gp))
geo_poly.close()
gp.close()

In [6]:
print(f"df_death_rate: {df_death_rate.shape}\ndf_Nmw: {df_Nmw.shape}\ndf_service: {df_service.shape}\ndf_service_common: {df_service_common.shape}\ndf_medicion: {df_medicion.shape}\ndf_welfare: {df_welfare.shape}")

df_death_rate: (18507, 8)
df_Nmw: (1829, 7)
df_service: (3192, 5)
df_service_common: (3788, 11)
df_medicion: (4724, 5)
df_welfare: (893, 9)


### 통합 데이터 생성
앞서 사용한 데이터들을 집계 함수를 이용해 하나의 데이터로 생성합니다.
#### 1. 연간 평균 사망률

In [7]:
df_corr = pd.DataFrame(df_death_rate.groupby(["국가", "연도"])["사망률"].mean()).reset_index().copy()
df_corr.columns = ["국가", "연도", "평균 사망률"]
df_corr

,국가,연도,평균 사망률
0,그리스,1998,40.600000
1,그리스,1999,40.382857
2,그리스,2000,40.357143
3,그리스,2001,39.397143
4,그리스,2002,38.622857
...,...,...,...
523,헝가리,2013,56.522857
524,헝가리,2014,55.531429
525,헝가리,2015,56.282857
526,헝가리,2016,53.828571


#### 2. 평균 의료 종사자 수
치과의사와 약사를 제외한, 의사, 간호사들만 이용합니다

In [8]:
df_Nmw = df_Nmw[df_Nmw["직업"].isin(["의사", "간호사"])].reset_index(drop=True).copy()
df_Nmw.sample(5)

,국가,연도,직업,수,국가영문,위도,경도
668,노르웨이,2003,의사,14746.0,norway,62.000000,10.000000
526,이탈리아,2003,간호사,300365.0,italy,42.833333,12.833333
43,한국,2019,간호사,410694.0,korea,37.000000,127.500000
326,핀란드,2008,간호사,69818.0,finland,64.000000,26.000000
588,라트비아,2011,간호사,10206.0,latvia,57.000000,25.000000


평균 의료 종사자 수를 구합니다

In [9]:
temp = pd.DataFrame(df_Nmw.groupby(["국가", "연도"])["수"].mean()).reset_index().copy()
temp.columns = ["국가", "연도", "평균 의료 인력 수"]
temp.sample(5)

,국가,연도,평균 의료 인력 수
314,영국,2001,306748.5
42,노르웨이,2006,44435.0
390,이탈리아,2012,267409.0
478,핀란드,2003,13365.0
141,라트비아,2017,7548.5


평균 사망률과 평균 의료 인력을 합칩니다.

In [10]:
df_corr = pd.merge(left=df_corr, right=temp, on=["국가", "연도"]).copy()
df_corr

,국가,연도,평균 사망률,평균 의료 인력 수
0,그리스,1999,40.382857,28452.0
1,그리스,2000,40.357143,29704.0
2,그리스,2001,39.397143,32068.0
3,그리스,2002,38.622857,36063.0
4,그리스,2003,38.517143,36319.0
...,...,...,...,...
449,헝가리,2013,56.522857,47668.5
450,헝가리,2014,55.531429,48040.0
451,헝가리,2015,56.282857,47079.5
452,헝가리,2016,53.828571,47336.5


#### 3. 보건 관련 지출비
나라 간의 개인 생활 수준을 비교하기엔 PPP가 적합하므로 여러 항목 중 **1인당 보건지출(PPP)(US%)** 항목만을 이용합니다

In [11]:
temp = pd.DataFrame(df_service.groupby(["국가별", "연도", "항목"])["데이터값"].mean()).reset_index()
temp

,국가별,연도,항목,데이터값
0,그리스,2000,1인당 보건지출(PPP)(US$),1417.777954
1,그리스,2000,1인당 보건지출(US$),885.410400
2,그리스,2000,경상(GDP 대비%),7.243326
3,그리스,2000,공공(경상보건비 대비%),61.626427
4,그리스,2000,본인부담(경상보건비 대비%),36.272114
...,...,...,...,...
3187,헝가리,2018,1인당 보건지출(US$),1081.802856
3188,헝가리,2018,경상(GDP 대비%),6.700072
3189,헝가리,2018,공공(경상보건비 대비%),69.100578
3190,헝가리,2018,본인부담(경상보건비 대비%),26.914713


In [12]:
temp = temp[temp["항목"]=="1인당 보건지출(PPP)(US$)"].reset_index(drop=True).copy()
temp.columns = ["국가", "연도", "항목", "1인당 보건지출"]
temp = temp[["국가", "연도", "1인당 보건지출"]].copy()
temp

,국가,연도,1인당 보건지출
0,그리스,2000,1417.777954
1,그리스,2001,1678.575684
2,그리스,2002,1864.497559
3,그리스,2003,2019.099609
4,그리스,2004,2061.648193
...,...,...,...
527,헝가리,2014,1808.397217
528,헝가리,2015,1834.953613
529,헝가리,2016,1949.052490
530,헝가리,2017,2003.747681


In [13]:
df_corr = pd.merge(left=df_corr, right=temp, on=["국가", "연도"]).copy()
df_corr

,국가,연도,평균 사망률,평균 의료 인력 수,1인당 보건지출
0,그리스,2000,40.357143,29704.0,1417.777954
1,그리스,2001,39.397143,32068.0,1678.575684
2,그리스,2002,38.622857,36063.0,1864.497559
3,그리스,2003,38.517143,36319.0,2019.099609
4,그리스,2004,37.840000,36133.0,2061.648193
...,...,...,...,...,...
390,헝가리,2012,58.845714,46691.5,1722.257812
391,헝가리,2013,56.522857,47668.5,1773.085571
392,헝가리,2014,55.531429,48040.0,1808.397217
393,헝가리,2015,56.282857,47079.5,1834.953613


#### 4. 의약품 판매 소비  


In [14]:
temp = pd.DataFrame(df_medicion.groupby(["국가", "연도"])[["의약품소비량", "의약품판매량"]].mean()).reset_index()
temp.columns = ["국가", "연도", "평군 소비량", "평균 판매량"]
temp

,국가,연도,평군 소비량,평균 판매량
0,그리스,1998,29.609091,2032.8
1,그리스,1999,32.909091,2366.4
2,그리스,2000,35.681818,2398.1
3,그리스,2001,37.318182,2807.4
4,그리스,2002,40.245455,3557.4
...,...,...,...,...
496,헝가리,2016,46.436364,1772.0
497,헝가리,2017,47.063636,1796.5
498,헝가리,2018,48.454545,2240.1
499,헝가리,2019,50.290909,2183.7


In [15]:
df_corr = pd.merge(left=df_corr, right=temp, on=["국가", "연도"]).copy()
df_corr

,국가,연도,평균 사망률,평균 의료 인력 수,1인당 보건지출,평군 소비량,평균 판매량
0,그리스,2000,40.357143,29704.0,1417.777954,35.681818,2398.1
1,그리스,2001,39.397143,32068.0,1678.575684,37.318182,2807.4
2,그리스,2002,38.622857,36063.0,1864.497559,40.245455,3557.4
3,그리스,2003,38.517143,36319.0,2019.099609,43.709091,4897.5
4,그리스,2004,37.840000,36133.0,2061.648193,47.118182,6218.8
...,...,...,...,...,...,...,...
305,헝가리,2012,58.845714,46691.5,1722.257812,41.963636,1974.9
306,헝가리,2013,56.522857,47668.5,1773.085571,42.781818,1923.0
307,헝가리,2014,55.531429,48040.0,1808.397217,44.318182,1904.3
308,헝가리,2015,56.282857,47079.5,1834.953613,44.972727,1669.7


#### 5. 보건 서비스와 공공사회
##### 보건 서비스

In [16]:
pd.Series(df_service_common["항목구분"].unique())

0         보건 당기 지출
1       치료 및 재활 치료
2            장기 치료
3        보건 보조 서비스
4              의료품
5             예방치료
6    정부의료체계 및 재정행정
dtype: object

보건 서비스 지출 내역에서, **치료 및 재활 치료, 장기 치료, 예방치료** 항목만 사용합니다.  
보건 보조 서비스는 인력, 정부의료체계 및 재정행정은 공공사회와 연관이 있고 의료품은 앞서 데이터를 가지고 있기 때문입니다.

In [17]:
temp = df_service_common[df_service_common["항목구분"].isin(["치료 및 재활 치료", "장기 치료", "예방치료"])].reset_index(drop=True)

In [18]:
temp = temp.groupby(["국가", "연도"])[["서비스비용(백만$)"]].mean().reset_index()
temp.columns = ["국가", "연도", "평균 치료비"]
temp

,국가,연도,평균 치료비
0,그리스,2003,506.273333
1,그리스,2004,575.290000
2,그리스,2005,625.670000
3,그리스,2006,648.686667
4,그리스,2007,769.300000
...,...,...,...
537,헝가리,2015,1701.903333
538,헝가리,2016,1776.880000
539,헝가리,2017,1956.316667
540,헝가리,2018,2131.090000


In [20]:
df_corr = pd.merge(left=df_corr, right=temp, on=["국가", "연도"]).copy()
df_corr

,국가,연도,평균 사망률,평균 의료 인력 수,1인당 보건지출,평군 소비량,평균 판매량,평균 치료비
0,그리스,2003,38.517143,36319.0,2019.099609,43.709091,4897.5,506.273333
1,그리스,2004,37.840000,36133.0,2061.648193,47.118182,6218.8,575.290000
2,그리스,2005,36.391429,36666.0,2194.105469,31.100000,6939.4,625.670000
3,그리스,2006,35.162857,35797.0,2358.683105,36.900000,7703.6,648.686667
4,그리스,2007,34.914286,35658.0,2462.738281,38.900000,10673.5,769.300000
...,...,...,...,...,...,...,...,...
287,헝가리,2012,58.845714,46691.5,1722.257812,41.963636,1974.9,1827.263333
288,헝가리,2013,56.522857,47668.5,1773.085571,42.781818,1923.0,1946.356667
289,헝가리,2014,55.531429,48040.0,1808.397217,44.318182,1904.3,1968.843333
290,헝가리,2015,56.282857,47079.5,1834.953613,44.972727,1669.7,1701.903333


##### 공공 사회 복지 지출
`G`는 10억을 나타내는 SI 접두어 입니다.

In [24]:
temp = pd.DataFrame(df_welfare.groupby(["국가", "연도"])["복지비용(10억$)"].mean()).reset_index()
temp.columns = ["국가", "연도", "평균 복지 비용(G$)"]
temp

,국가,연도,평균 복지 비용(G$)
0,그리스,1998,2527.00
1,그리스,1999,2565.00
2,그리스,2000,2315.78
3,그리스,2001,2533.32
4,그리스,2002,2829.92
...,...,...,...
800,헝가리,2015,2552.04
801,헝가리,2016,2608.55
802,헝가리,2017,2817.10
803,헝가리,2018,3015.52


In [25]:
df_corr = pd.merge(left=df_corr, right=temp, on=["국가", "연도"]).copy()
df_corr

,국가,연도,평균 사망률,평균 의료 인력 수,1인당 보건지출,평군 소비량,평균 판매량,평균 치료비,평균 복지 비용(G$)
0,그리스,2003,38.517143,36319.0,2019.099609,43.709091,4897.5,506.273333,3674.58
1,그리스,2004,37.840000,36133.0,2061.648193,47.118182,6218.8,575.290000,4401.15
2,그리스,2005,36.391429,36666.0,2194.105469,31.100000,6939.4,625.670000,4832.10
3,그리스,2006,35.162857,35797.0,2358.683105,36.900000,7703.6,648.686667,5302.02
4,그리스,2007,34.914286,35658.0,2462.738281,38.900000,10673.5,769.300000,6401.85
...,...,...,...,...,...,...,...,...,...
287,헝가리,2012,58.845714,46691.5,1722.257812,41.963636,1974.9,1827.263333,2904.10
288,헝가리,2013,56.522857,47668.5,1773.085571,42.781818,1923.0,1946.356667,3005.88
289,헝가리,2014,55.531429,48040.0,1808.397217,44.318182,1904.3,1968.843333,3027.20
290,헝가리,2015,56.282857,47079.5,1834.953613,44.972727,1669.7,1701.903333,2552.04


여러 데이터의 공통 항목들로 평균 값을 담고 있는데 데이터 프레임을 생성했습니다.